In [76]:
import requests
from bs4 import BeautifulSoup
import gzip
import functools

In [77]:
def get_soup(html_str, parser='html.parser'):
    return BeautifulSoup(html_str, parser)

In [78]:
BASE = "https://www.allrecipes.com/"
TEMP = 'temp.gz'

In [79]:
def get_request(endpoint):
    return requests.get(endpoint)

In [80]:
gs_index = get_soup(get_request('%sgsindex.xml'%BASE).text, parser='xml')

In [81]:
sitemaps = [s.loc.contents[0] for s in gs_index.find_all('sitemap')]

In [82]:
# TEMP FILE FOR INSPECTION IN VSCODE
sitemap = get_request(sitemaps[0])

with open(TEMP, 'wb') as f:
    f.write(sitemap.content)

with gzip.open(TEMP, 'rb') as f:
    file_content = f.read()
    with open("text.xml", "wb") as b:
        b.write(file_content)

In [83]:
def get_sitemap_xml(sitemap):
    sitemap = get_request(sitemap)
    with open(TEMP, 'wb') as f:
        f.write(sitemap.content)
    with gzip.open(TEMP, 'rb') as f:
        file_content = f.read()
    return get_soup(file_content, parser='xml')
    
        

In [86]:
def get_recipes_from_xml(sitemap):
    sitemap1 = get_sitemap_xml(sitemap)
    url_tags = sitemap1.find_all('url')
    return [e.loc.contents[0] for e in url_tags]

In [87]:
sitemaps_excluded = list(sitemaps)
sitemaps_excluded.pop(4)
recipes = [get_recipes_from_xml(sitemap) for sitemap in sitemaps_excluded]

In [88]:
recipe_list = functools.reduce(lambda a,b: a+b, recipes)

In [89]:
recipe_list

['https://www.allrecipes.com/recipe/6663/crispy-cheese-twists/',
 'https://www.allrecipes.com/recipe/6664/basil-roasted-peppers-and-monterey-jack-cornbread/',
 'https://www.allrecipes.com/recipe/6665/moms-yeast-rolls/',
 'https://www.allrecipes.com/recipe/6666/sweet-potato-bread-i/',
 'https://www.allrecipes.com/recipe/6667/cornish-splits/',
 'https://www.allrecipes.com/recipe/6668/orange-buns/',
 'https://www.allrecipes.com/recipe/6669/jalapeno-cheese-bread/',
 'https://www.allrecipes.com/recipe/6670/dees-health-bread/',
 'https://www.allrecipes.com/recipe/6671/oatmeal-bread-i/',
 'https://www.allrecipes.com/recipe/6672/strawberry-bread-i/',
 'https://www.allrecipes.com/recipe/6673/sun-dried-tomato-and-asiago-cheese-bread/',
 'https://www.allrecipes.com/recipe/6674/hawaiian-sweet-bread/',
 'https://www.allrecipes.com/recipe/6675/the-best-corn-bread-youll-ever-eat/',
 'https://www.allrecipes.com/recipe/6676/no-knead-refrigerator-rolls/',
 'https://www.allrecipes.com/recipe/6677/sourdou

In [98]:
soup = get_soup(get_request('https://www.allrecipes.com/recipe/7038/panettone-ii/').text)

In [116]:
def get_v1(soup):
    title = soup.find(id="recipe-main-content").text
    rating=float(soup.find("div",class_="rating-stars")['data-ratingstars'])
    review_count=int(soup.find("span",class_="review-count").text.split()[0])
    ingredients = []
    try:
        for item in filter(lambda x: x['itemprop'] =='recipeIngredient' ,soup.find_all('span',class_="recipe-ingred_txt")):
            ingredients.append(item.text)
    except Exception:
        pass
    directions = [e.text.strip() for e in soup.find_all('span', class_='recipe-directions__list--item')[:-1] ]
    data={'title':title, 'rating': rating, 'review_count': review_count, "ingredients":ingredients,"directions":directions}
    return data

In [118]:
def func(x):
    try:
        soup = get_soup(get_request(x).text)
        print(get_v1(soup))
        return get_v1(soup)
    except Exception:
        return  None
list(map(func, recipe_list[0:100]))

{'title': 'Crispy Cheese Twists', 'rating': 4.17333316802979, 'review_count': 51, 'ingredients': ['1/2 cup Parmesan cheese', '3/4 teaspoon ground black pepper', '1/2 teaspoon garlic powder', '1 (17.5 ounce) package frozen puff pastry, thawed', '1 egg white'], 'directions': ['Combine parmesan cheese, pepper and garlic powder. Unfold pastry sheets onto cutting board.  Brush lightly with egg white; sprinkle each sheet with 1/4 of the cheese mixture.  Lightly press into pastry, turn over; repeat.  Cut each sheet into 12 (1-inch) strips; twist.', 'Place on ungreased cookie sheet and bake in 350 degrees F (175 degrees C) oven for 15 minutes or until golden brown.']}
{'title': 'Basil, Roasted Peppers and Monterey Jack Cornbread', 'rating': 4.32758617401123, 'review_count': 47, 'ingredients': ['1/2 cup unsalted butter, chilled and cubed', '1 cup chopped onion', '1 3/4 cups cornmeal', '1 1/4 cups all-purpose flour', '1/4 cup white sugar', '1 tablespoon baking powder', '1 1/2 teaspoons salt', '1

{'title': 'Oatmeal Bread I', 'rating': 4.60526323318481, 'review_count': 30, 'ingredients': ['1 cup rolled oats', '1/2 cup molasses', '1/3 cup vegetable oil', '1 teaspoon salt', '1 1/2 cups boiling water', '2 tablespoons active dry yeast', '1/2 cup lukewarm water (100 degrees F/38 degrees C)', '1 cup whole wheat flour', '5 cups bread flour', '2 eggs'], 'directions': ['Combine oats, molasses, oil, salt and boiling water. Let cool to about 105 degrees F.', 'Dissolve the yeast in the warm water and let stand for 5 minutes or until creamy. Stir the yeast into oat mixture and mix well. Add whole wheat flour, 2 cups bread flour, and the eggs. Mix until well combined.', 'Stir in enough of the remaining flour to make a soft dough. Turn dough out to a floured counter and knead for about 10 minutes. Place the dough in a well-greased bowl and cover with greased plastic wrap. Let the dough rest in the refrigerator overnight.', 'Preheat oven to 375 degrees F (190 degrees C). Grease 3 regular loaf p

{'title': 'Onion Bread I', 'rating': 4.5, 'review_count': 21, 'ingredients': ['2 (.25 ounce) packages active dry yeast', '3/4 cup warm water', '2 cups milk', '3 tablespoons white sugar', '2 1/2 teaspoons salt', '3 tablespoons margarine', '6 1/4 cups all-purpose flour', '1 (1 ounce) package dry onion soup mix'], 'directions': ['Warm the milk in a small saucepan until it bubbles, then remove from heat. Let cool until lukewarm. In a large bowl, stir together yeast, warm water, and 1 teaspoon sugar.  Set aside to proof.', 'Add milk, 2 2/3 tablespoons sugar, salt, butter or margarine, flour, and soup mix to the yeast in the bowl.  Combine until dough forms, and then turn out on a lightly floured surface.   Knead until dough is elastic.  Oil a large bowl.  Place the dough in the bowl, and turn several times to coat.  Cover the bowl with a damp cloth, and set aside in a warm place to rise for 30 minutes.', 'Divide dough in half.  Shape into loaves, and place into two greased 8 1/2 x 4 1/2 inc

{'title': 'Buttermilk Bread I', 'rating': 3.8139533996582, 'review_count': 41, 'ingredients': ['1 1/2 cups buttermilk', '1 1/2 tablespoons margarine', '2 tablespoons white sugar', '1 teaspoon salt', '3 cups bread flour', '1 1/3 cups whole wheat flour', '2 1/4 teaspoons active dry yeast'], 'directions': ['Place buttermilk, butter or margarine, sugar, salt, flour, whole wheat flour, and yeast into pan of bread machine.', 'Bake on White Bread setting.  Cool on wire racks before slicing.']}
{'title': "Michell Jenny's Challah", 'rating': 4.74226808547974, 'review_count': 79, 'ingredients': ['7/8 cup water', '1/2 tablespoon salt', '1/4 cup honey', '2 eggs', '1/4 cup butter, melted', '4 cups all-purpose flour', '2 teaspoons active dry yeast'], 'directions': ['Place water, salt, honey, eggs, melted butter or margarine, flour, and yeast in order given into the bread machine. Set the machine to Dough cycle, and start.', 'Remove dough from the machine after first rising. Divide dough into three o

{'title': "Nick's Favorite Cherry Pecan Bread", 'rating': 4.32258081436157, 'review_count': 29, 'ingredients': ['1/2 cup margarine', '3/4 cup white sugar', '2 eggs', '1 cup buttermilk', '1 teaspoon baking soda', '1/2 teaspoon salt', '1 teaspoon vanilla extract', '2 cups all-purpose flour', '1 (10 ounce) jar maraschino cherries, drained and chopped', '1 cup chopped pecans'], 'directions': ['Preheat oven to 350 degrees F (175 degrees C).  Grease a 9 x 5 inch loaf pan.', 'In a large bowl, cream butter or margarine and sugar together.  Add eggs, and continue mixing until light and fluffy.  Add baking soda, salt, and vanilla; mix thoroughly.  Alternately add buttermilk and flour to the creamed mixture, mixing only enough to hold ingredients together. Fold in cherries and nuts, mixing lightly.  Turn batter into prepared pan.', 'Bake for 50 to 60 minutes, or until bread tests done.  Cool completely before slicing.']}
{'title': 'Alienated Blueberry Muffins', 'rating': 3.97304582595825, 'review

...

{'title': 'Poppy Seed Bread I', 'rating': 4.51612901687622, 'review_count': 20, 'ingredients': ['1 teaspoon salt', '1/4 cup poppy seeds', '3 cups all-purpose flour', '4 eggs', '1 1/2 cups vegetable oil', '2 cups white sugar', '1 teaspoon baking soda', '1 teaspoon vanilla extract', '1 cup milk'], 'directions': ['Preheat oven to 325 degrees F (165 degrees C).  Grease and flour two 9 x 5 inch loaf pans.', 'Mix the flour, salt, sugar, and soda in a mixing bowl.  Add vanilla, milk, eggs, and oil.  Beat for five minutes with an electric mixer.   Stir in the poppy seeds.  Pour batter into loaf pans.', 'Bake for one hour.  Cool on wire racks.']}
{'title': 'Zucchini Bread', 'rating': 4.49253749847412, 'review_count': 287, 'ingredients': ['3 cups all-purpose flour', '3 eggs', '2 cups white sugar', '1 cup vegetable oil', '2 cups grated zucchini', '1 cup semisweet chocolate chips', '1 teaspoon ground cinnamon', '1 teaspoon baking soda', '1/4 teaspoon baking powder', '1/2 cup sour cream'], 'directi

{'title': 'Tomato Bread I', 'rating': 4.625, 'review_count': 7, 'ingredients': ['1 cup tomato juice', '1 cup water', '1 (.25 ounce) package instant yeast', '1/4 cup vegetable oil', '1/3 cup honey', '1/4 cup chopped fresh parsley', '1/4 cup chopped green onions', '2 cloves garlic', '1 carrot, shredded', '1 teaspoon salt', '6 cups bread flour'], 'directions': ['In a sauce pan, heat the tomato juice and water over a low heat until warm to the touch.  Pour into a large warmed bowl, and add yeast and honey; stir to dissolve yeast.  Allow to rest until yeast is creamy.', 'Mix in oil, parsley, onion, garlic, carrot, and salt.  Add 1 cup of the flour, and stir until smooth. Add more flour, until a firm dough is formed.  Knead five minutes on a lightly floured surface.  Place dough in a greased bowl, and turn to coat the surface completely.  Allow to rise in a warm place until doubled in size.', 'Punch down, and divide into halves.  Form two loaves, and put into greased 9 x 5 inch loaf pans.  A

{'title': 'Lower Fat Banana Nut Bread', 'rating': 4.1304349899292, 'review_count': 17, 'ingredients': ['3/4 cup fat free sour cream', '4 egg whites', '1/4 cup chopped walnuts', '1/3 cup raisins (optional)', '2 teaspoons baking powder', '2 teaspoons baking soda', '6 very ripe bananas, mashed', '2 tablespoons reduced fat margarine', '1 tablespoon vanilla extract', '1 teaspoon salt', '2/3 cup packed light brown sugar', '4 cups unbleached all-purpose flour'], 'directions': ['Preheat oven to 375 degrees F (190 degrees C). Lightly spray two 8x4x2 inch loaf pans with a non-stick cooking spray.', 'Combine the fat free sour cream, egg whites, vanilla, bananas and margarine and mix on medium speed of electric mixer until smooth, creamy and well blended.', 'Sift the flour, baking powder, baking soda, salt and light brown sugar into the banana mixture. Stir with a spoon until combined. Add more flour if necessary until a thick and rather resistant dough is formed. Fold in the optional nuts and rai

{'title': 'Sourdough Bread II', 'rating': 3.46666669845581, 'review_count': 12, 'ingredients': ['2 1/4 teaspoons active dry yeast', '1/2 cup warm water (110 degrees F/45 degrees C)', '2 cups bread flour', '2 cups warm water', '1 tablespoon white sugar', '1 teaspoon salt', '2 1/4 teaspoons active dry yeast', '1/2 cup warm water (110 degrees F/45 degrees C)', '1 1/2 cups sourdough starter', '2 teaspoons white sugar', '1 1/2 teaspoons salt', '1 1/2 teaspoons baking soda', '5 1/2 cups bread flour', '1 tablespoon butter, melted'], 'directions': ['In a medium bowl, dissolve 2 1/4 teaspoons yeast in 1/2 cup warm water. Stir in 2 cups flour, 2 cups lukewarm water, 1 tablespoon sugar, and 1 teaspoon salt.  Beat smooth with rotary beater. Let stand 3 to 5 days uncovered at room temperature stirring 2 or 3 times a day. Cover at night to prevent drying.', 'In a large bowl, dissolve 2 1/4 teaspoons yeast in 1/2 cup warm water. Blend in starter batter, 2 teaspoons sugar, and 1 1/2 teaspoons salt.  A

KeyboardInterrupt: 